In [1]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import re

In [2]:
with open("./az_papers/paper-structure.dtd") as f:
    dtd_soup = BeautifulSoup(f, 'html.parser')

FileNotFoundError: [Errno 2] No such file or directory: './az_papers/paper-structure.dtd'

In [25]:
# parse data structure
structure = {}

elem_name = ""
for line in tqdm(dtd_soup):
    line = line.string
    split_by_space = line.split()
    if len(split_by_space):
        if split_by_space[0] == "ATTLIST":
            elem_name = split_by_space[1]
            split_by_line = line.split("\n")
            if len(split_by_line) > 1:
                structure[elem_name] = [attr_line.split()[0] for attr_line in split_by_line]
            else:
                structure[elem_name] = [split_by_line[0].split()[2]]


100%|██████████| 58/58 [00:00<00:00, 72748.10it/s]


In [ ]:
structure

In [2]:
def clean_bs_text(line):
    """
    Returns bs datatype, not string!
    """
    if line.eqn:
        for eq in line.find_all("eqn"):
            eq.replace_with("EQN")
    if line.cref:
        for ref in line.find_all("cref"):
            ref.replace_with("CREF")

    replaced_str = line.get_text()
    replaced_str = re.sub("``", '"', replaced_str)
    replaced_str = re.sub("''", '"', replaced_str)
    line.string = replaced_str

    return line

In [3]:

from spacy.lang.en import English

nlp = English()
nlp.add_pipe("sentencizer")

In [119]:
def split_sents_get_labels(combined_sent):
    doc = nlp(combined_sent["text"])
    out_sents = []
    for sent in doc.sents:
        out_sents.append(
            {
            "text": sent.text,
            "label": combined_sent["label"],
            "s_id": combined_sent['s_id'],
            "header": combined_sent['header'],
            "fileno": combined_sent['fileno'],
            }
            )
    return out_sents

In [29]:
LINE_DIV = "="*100+"\n"

In [30]:
print(LINE_DIV)

In [168]:
# parse all data

docs = []

print("Processing all az-scixml files...")
 



Processing all az-scixml files...


  4%|▎         | 3/82 [00:00<00:03, 23.18it/s]

hello
hello


 11%|█         | 9/82 [00:00<00:04, 14.92it/s]

hello
hello
hello
hello
hello
hello
hello


...

hello
hello
hello
hello
hello
hello
hello
hello
hello


 87%|████████▋ | 71/82 [00:03<00:00, 28.78it/s]

hello
hello
hello
hello
hello
hello


100%|██████████| 82/82 [00:03<00:00, 24.51it/s]

hello
hello
hello
hello
# of no label sentences:  2


In [91]:
len(docs)

79

In [149]:
# test one paper
no_label = 0

doc = {}
with open(os.path.join(".", "az_papers", "raw", "9405010.az-scixml")) as f:
    soup = BeautifulSoup(f, 'html.parser', from_encoding='iso-8859-1')
    #soup = BeautifulSoup(f, features="xml", from_encoding='iso-8859-1')
    soup = soup.paper
    doc['title'] = soup.title.text
    doc['year'] = soup.year.text
    doc['fileno'] = soup.fileno.text
    doc['classification'] = soup.classification.text
    doc['abstract'] = []

    abstr = soup.abstract
    for line in abstr.find_all('a-s'):
        line = clean_bs_text(line)
        sent_dict = {'text': line.get_text(), 'label': line.get('az')}
        doc['abstract'].append(sent_dict)
    
    doc['body'] = []
    # doc['body'] = [
    # {'header':..., 'sentences': [{'text': ..., "label": ...}, {...}]}, {...}
    # ]
    body = soup.body
    for div in body.find_all("div"):
        header = ""
        if int(div.get("depth")) > 1: # update header
            header = div.header.get_text()
            continue
        header = header if header else div.header.get_text()
        for parag in div.find_all('p'):
            body_dict = {}
            body_dict['header'] = header
            body_dict['sentences'] = []

            combined_sent = {"text": "", "label": "", "header": header, "fileno": doc["fileno"],}
            for i, line in enumerate(parag.find_all('s')):
                s_id = line.get("id")
                combined_sent['s_id'] = s_id
                combined_sent['header'] = header
                line = clean_bs_text(line)
                if line.get("type")=="ITEM":
                    label = line.get('az')
                    if not label:
                        no_label += 1
                        continue
                    if combined_sent["label"]:
                        if combined_sent["label"] != label:
                            combined_sent = split_sent_get_label(combined_sent)
                            body_dict['sentences'].extend(combined_sent) # save previous aggregated sents
                            combined_sent = {"text": line.get_text(), "label": label, "s_id": s_id, "header": header, "fileno": doc["fileno"], }
                            continue
                        else:
                            pass # sents with the same label are lumped as one big sentence. 
                            # this is dealt with in the next elif.
                    else:
                        combined_sent["label"] = label # first sent in the ITEM list
                    combined_sent["text"] = "".join([combined_sent["text"], line.get_text()])
                    combined_sent["s_id"] = s_id

                    if (i==(len(parag.find_all('s'))-1)) and combined_sent["text"]: # the final item is the final line in the parag
                        print("hello")
                        combined_sent = split_sents_get_labels(combined_sent)
                        body_dict['sentences'].extend(combined_sent)
                        combined_sent = {"text": "", "label": "", "header": header, "fileno": doc["fileno"]} # reset
                    continue
                    

                elif (line.get("type")!="ITEM") and combined_sent["text"]: # we're out of an ITEM list but still in a paragraph
                    print("hi")
                    combined_sent = split_sents_get_labels(combined_sent)
                    body_dict['sentences'].extend(combined_sent)
                    combined_sent = {"text": "", "label": "", "header": header, "fileno": doc["fileno"],} # reset

                label = line.get('az') # deal with current line
                if not label: 
                    no_label += 1
                    continue
                sent_dict = {'text': line.get_text(), 'label': label, "header": header, "s_id": s_id, "fileno": doc["fileno"],}
                body_dict['sentences'].append(sent_dict)
            if body_dict['sentences']:
                if parag.find_all('image'):
                    body_dict['sentences'][-1]['text'] = body_dict['sentences'][-1]['text'] + "[IMAGE]"
                doc['body'].append(body_dict)
print("# of no label sentences: ", no_label)



hi
hi
hi
# of no label sentences:  0


In [150]:
len(doc['body'])

51

In [154]:
doc['abstract'][0]

{'text': ' It is claimed that a variety of facts concerning ellipsis , event reference , and interclausal coherence can be explained by two features of the linguistic form in question : ',
 'label': 'AIM'}

In [121]:
import jsonlines

with jsonlines.open("az_papers_06062023.jsonl", "w") as f:
    f.write_all(docs)

In [122]:
az_papers = list(jsonlines.open("az_papers_06062023.jsonl"))
az_papers[0]['body'][0]

{'header': ' Introduction ',
 'sentences': [{'text': ' In the approach to discourse structure developed in Sidner 1983 and Grosz et al. 1986 , a discourse exhibits both global and local coherence . ',
   'label': 'OTH',
   'header': ' Introduction ',
   's_id': 'S-0',
   'fileno': '9410005'},
  {'text': ' On this view , a key element of local coherence is centering , a system of rules and constraints that govern the relationship between what the discourse is about and some of the linguistic choices made by the discourse participants , e.g. choice of grammatical function , syntactic structure , and type of referring expression ( proper noun , definite or indefinite description , reflexive or personal pronoun , etc . ) ',
   'label': 'OTH',
   'header': ' Introduction ',
   's_id': 'S-1',
   'fileno': '9410005'},
  {'text': ' Pronominalization in particular serves to focus attention on what is being talked about ; inappropriate use or failure to use pronouns causes communication to be le

In [2]:
import jsonlines

In [123]:
docs = list(jsonlines.open("az_papers_06062023.jsonl"))

az_abs = [it["abstract"] for it in docs]

with jsonlines.open("az_papers_abstract_06062023.jsonl", "w") as f:
    f.write_all(az_abs)

# doc['body'] = [
# {'header':..., 'sentences': [{'text': ..., "label": ...}, {...}]}, {...}
# ]
az_body = [parag['sentences'] for doc in docs for parag in doc["body"]]
with jsonlines.open("az_papers_body_06062023.jsonl", "w") as f:
    f.write_all(az_body)


In [124]:
az_all = az_abs + az_body
with jsonlines.open("az_papers_all_06062023.jsonl", "w") as f:
    f.write_all(az_all)

In [60]:
abst = list(jsonlines.open("az_papers_abstract.jsonl"))

In [64]:
az_body = [parag['sentences'] for doc in az_papers for parag in doc["body"]]
with jsonlines.open("az_papers_body.jsonl", "w") as f:
    f.write_all(az_body)

In [13]:
with open("az_papers/9504033.az-scixml") as f:
    soup = BeautifulSoup(f, 'html.parser', from_encoding="iso-8859-1")
    

In [15]:
for line in soup.find_all('s'):
    if line.ref:
        for r in line.find_all('ref'):
            print(line)
            r.replace_with("CREF")

<s az="BKG" id="S-1"> For instance , parsing noun compounds appears to require detailed world knowledge that is unavailable outside a limited domain <ref type="P">Sparck Jones 1983</ref> . </s>
